## Auslesen der Dateien ##

Achtung: Die Dateien aus dem ersten Verzeichnis (hier D:/hka-aqm-m) müssen extrahiert vorliegen (d.h. nicht als zip-Ordner).

hka-aqm-m

            > ... .dat

            > ... .dat

In [8]:
from Preprocessing_M_test import DataExtractor, DataPreprocessing

directory = r"C:\Users\maxda\Studium\Semester6\IoT\Gebäude_Daten"
new_directory = r"C:\Users\maxda\Studium\Semester6\IoT\Gebäude_Daten"
de = DataExtractor(first_directory=directory, new_directory=new_directory)

df = de.create_df()

Read data successfully.
Data contains 979166 data points and 18 columns.


## Preprocessing ##

df_preprocessed := Datensatz mit Wetterdaten und Jahreszeiten

df_preprocessed_without_weather := Datensatz ohne Wetterdaten, aber mit Jahreszeiten

df_preprocessed_without_seasons := Datensatz mit Wetterdaten, aber ohne Jahreszeiten

In [9]:
label = "CO2"

dp = DataPreprocessing(label = label)

df_preprocessed = dp.preprocess_df(df, rolling_window = "3d", sample_time = "1d")

Number of outliers detected: -73438
Number of normal samples detected: 905728


In [10]:
df_preprocessed.columns

Index(['date_time', 'tmp', 'hum', 'CO2', 'VOC', 'vis', 'IR', 'BLE', 'tavg',
       'tmin', 'tmax', 'prcp', 'wdir', 'wspd', 'wpgt', 'pres', 'room_number',
       'time_diff_sec', 'tmp_diff', 'hum_diff', 'VOC_diff', 'vis_diff',
       'IR_diff', 'BLE_diff', 'tavg_diff', 'tmin_diff', 'tmax_diff',
       'prcp_diff', 'wdir_diff', 'wspd_diff', 'wpgt_diff', 'pres_diff', 'year',
       'dayofweek', 'hour', 'season', 'color', 'VOC_CO2_ratio'],
      dtype='object')

In [11]:
df_preprocessed_without_weather = df_preprocessed.drop(columns = ['tavg',
       'tmin', 'tmax', 'wdir', 'wspd', 'wpgt', 'pres',  'tavg_diff', 'tmin_diff', 'tmax_diff', 'wdir_diff', 'wspd_diff',
       'wpgt_diff', 'pres_diff'], axis = 1)

df_preprocessed_without_seasons = df_preprocessed.drop(columns = ["season"], axis = 1)

## Feature Engineering ##

In [12]:
import pickle
fe = pickle.load(open("feature_engineerer.pickle", "rb"))

c:\Users\maxda\miniconda3\envs\IoT\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.4.2 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [13]:
from ML_Preparation.Feature_Engineering import *

n_steps = 7

# fe = FeatureEngineering(df_preprocessed,
#                         label = label, 
#                         categorical_features = ["season", "room_number", "dayofweek", "color"],
#                         automated_feature_engineering = False)

X_train, X_val, X_test, y_train, y_val, y_test = fe.feature_engineering(steps_to_forecast = n_steps, skip_scale = True)

'season'


In [14]:
fe_no_weather = FeatureEngineering(df_preprocessed_without_weather,
                        label = label, 
                        categorical_features = ["season", "room_number", "dayofweek", "color"],
                        automated_feature_engineering = False)

X_train_nowe, X_val_nowe, X_test_nowe, y_train_nowe, y_val_nowe, y_test_nowe = fe_no_weather.feature_engineering(steps_to_forecast = n_steps, skip_scale = True)

fe_no_seasons = FeatureEngineering(df_preprocessed_without_seasons,
                        label = label, 
                        categorical_features = ["room_number", "dayofweek", "color"],
                        automated_feature_engineering = False)

X_train_nose, X_val_nose, X_test_nose, y_train_nose, y_val_nose, y_test_nose = fe_no_seasons.feature_engineering(steps_to_forecast = n_steps, skip_scale = True)

In [15]:
fe.X_train.shape

(4603, 1, 68)

## Modelltraining ##

In [16]:
from Forecasting_Models import CO2_Forecasting_Model
from matplotlib import pyplot

trainierte Modelle laden

In [17]:
import tensorflow as tf
# from tensorflow.keras.models import load_model

train_models = False
label="CO2"
try:
    model = tf.keras.models.load_model(f"{label}_Forecasting_Model.keras")
    # model_no_weather = tf.keras.models.load_model(f"{label}_Forecasting_Model_no_weather.keras")
    # model_no_seasons = tf.keras.models.load_model(f"{label}_Forecasting_Model_no_seasons.keras")
except:
    print("Models couldn't be loaded. Train new models.")
    train_models = True

In [18]:
if train_models:    
    # Instantiate the model
    model = CO2_Forecasting_Model(n_steps, fe)

    history = model.model.fit(fe.X_train, fe.y_train, epochs=50, batch_size = 7, 
                                validation_data = (fe.X_val, fe.y_val),
                                verbose = 0, 
                                shuffle=False
                                )

    pyplot.plot(history.history['loss'], label='train')
    pyplot.plot(history.history["val_loss"], label = "val")
    pyplot.title("Training mit Jahreszeiten und Wetterdaten")
    pyplot.legend()
    pyplot.show()



    # model_no_weather = CO2_Forecasting_Model(n_steps, fe_no_weather)

    # history = model_no_weather.model.fit(fe_no_weather.X_train, fe_no_weather.y_train, epochs=50, batch_size=7, 
    #                                     validation_data = (fe_no_weather.X_val, fe_no_weather.y_val),
    #                                     verbose = 0, 
    #                                     shuffle=False
    #                                     )

    # pyplot.plot(history.history['loss'], label='train')
    # pyplot.plot(history.history["val_loss"], label = "val")
    # pyplot.title("Training mit Jahreszeiten und OHNE Wetterdaten")
    # pyplot.legend()
    # pyplot.show()

    # model_no_seasons = CO2_Forecasting_Model(n_steps, fe_no_seasons)

    # history = model_no_seasons.model.fit(fe_no_seasons.X_train, fe_no_seasons.y_train, epochs=50, batch_size = 7, 
    #                                     validation_data = (fe_no_seasons.X_val, fe_no_seasons.y_val),
    #                                     verbose = 0, 
    #                                     shuffle=False
    #                                 )

    # pyplot.plot(history.history['loss'], label='train')
    # pyplot.plot(history.history["val_loss"], label = "val")
    # pyplot.title("Training OHNE Jahreszeiten und mit Wetterdaten")
    # pyplot.legend()
    # pyplot.show()

In [19]:
fe.X_train.shape

(4603, 1, 68)

In [20]:
if train_models:
    model.model.save(f"{label}_Forecasting_Model.keras")
    model_no_weather.model.save(f"{label}_Forecasting_Model_no_weather.keras")
    model_no_seasons.model.save(f"{label}_Forecasting_Model_no_seasons.keras")

    pred = model.model.predict(fe.X_test)
    pred_no_weather = model_no_weather.model.predict(fe_no_weather.X_test)
    pred_no_seasons = model_no_seasons.model.predict(fe_no_seasons.X_test)
else:
    pred = model.predict(fe.X_test)
    # pred_no_weather = model_no_weather.predict(fe_no_weather.X_test)
    # pred_no_seasons = model_no_seasons.predict(fe_no_seasons.X_test)


40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step


## Modellevaluation ##

In [21]:
import plotly.express as px
import seaborn as sns
from Evaluator import Evaluator


ev = Evaluator()

for col in range(0, fe.y_test.shape[1]):
    print(f"t +", col)
    print("Modell mit Wetterdaten und Jahreszeiten (rot)")
    ev.evaluate(y_true = fe.y_test[:, col], y_pred = pred[:, col])
    # print("Modell ohne Wetterdaten und mit Jahreszeiten (grün)")
    # ev.evaluate(y_true = fe_no_weather.y_test[:, col], y_pred = pred_no_weather[:, col])
    # print("Modell mit Wetterdaten und ohne Jahreszeiten (violett)")
    # ev.evaluate(y_true = fe_no_seasons.y_test[:, col], y_pred = pred_no_seasons[:, col])

px.line(y = [fe.y_test[:, col], pred[:, col]], #pred_no_weather[:,col], pred_no_seasons[:, col]], 
            labels = {"wide_variable_0": "y_true",
                    "wide_variable_1": "Modell 1 - Vorhersage (Wetterdaten + Jahreszeiten)", 
                   "wide_variable_2": "Modell 2 - Vorhersage (Jahreszeiten)",
                   "wide_variable_3": "Modell 3 - Vorhersage (Wetterdaten)", },             
            title = f"Prediction for CO2(t + {col})  blue = y_true, red = y_pred").show()


t + 0
Modell mit Wetterdaten und Jahreszeiten (rot)
MSE:  7851.78
RMSE:  88.61
MAPE:  0.15
R2:  -0.5


t + 1
Modell mit Wetterdaten und Jahreszeiten (rot)
MSE:  7665.58
RMSE:  87.55
MAPE:  0.14
R2:  -0.46


t + 2
Modell mit Wetterdaten und Jahreszeiten (rot)
MSE:  7599.98
RMSE:  87.18
MAPE:  0.15
R2:  -0.45


t + 3
Modell mit Wetterdaten und Jahreszeiten (rot)
MSE:  7584.57
RMSE:  87.09
MAPE:  0.14
R2:  -0.45


t + 4
Modell mit Wetterdaten und Jahreszeiten (rot)
MSE:  7199.34
RMSE:  84.85
MAPE:  0.14
R2:  -0.37


t + 5
Modell mit Wetterdaten und Jahreszeiten (rot)
MSE:  7573.57
RMSE:  87.03
MAPE:  0.14
R2:  -0.44


t + 6
Modell mit Wetterdaten und Jahreszeiten (rot)
MSE:  7461.94
RMSE:  86.38
MAPE:  0.14
R2:  -0.4




## Deployment bzw. Inferenz mit dem trainierten Modell ##

In [22]:
from ML_Deployment import *

fe.df = df.copy()

deployer = Predictor(data = df, feature_engineering_class_object = fe, label = "CO2", is_forecast = True, roll = True, steps_to_forecast = 2)


'color'


In [23]:
deployer.x.shape

(6403, 1, 68)

In [24]:
if train_models:
    forecasted_pred = deployer.predict(x = deployer.x, model = model.model)
else:
    forecasted_pred = deployer.predict(x = deployer.x, model = model)

forecasted_pred

201/201 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step


,prediction_t+0,prediction_t+1,prediction_t+2,prediction_t+3,prediction_t+4,prediction_t+5,prediction_t+6,room_number
date_time,,,,,,,,
2022-06-01,523.866516,521.641113,523.778992,522.481995,520.589294,524.216431,524.368164,m209
2022-06-02,523.866516,521.641113,523.778992,522.481995,520.589294,524.216431,524.368164,m209
2022-06-03,523.866516,521.641113,523.778992,522.481995,520.589294,524.216431,524.368164,m209
2022-06-04,523.866516,521.641113,523.778992,522.481995,520.589294,524.216431,524.368164,m209
2022-06-20,527.532288,525.818359,528.218750,526.129700,524.416565,527.163147,527.119507,m209
...,...,...,...,...,...,...,...,...
2023-09-26,527.532288,525.818359,528.218750,526.129639,524.416504,527.163208,527.119446,m209
2023-09-26,527.532288,525.818359,528.218750,526.129639,524.416504,527.163208,527.119446,m209
2023-09-26,527.532288,525.818420,528.218750,526.129700,524.416565,527.163208,527.119507,m209


In [25]:
forecasted_pred.room_number.unique()

array(['m209'], dtype=object)